In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import ase
from ase.neighborlist import natural_cutoffs, NeighborList
from ase.io.trajectory import Trajectory
from ase.io import read as ase_read

import os
import json
import sys
sys.path.append("../py_src/")

import hydrogen_tracing
import config_plot

import re

In [ ]:
traj_path = '../test_data/221220_short_traj/rutile-011-nd-0/out.lammpstrj'
trajectory = ase_read(traj_path, index='1000:10000:10')

In [ ]:
dirname = os.path.dirname(traj_path)
dirname = '../test_data/230223_short_traj_publike/rutile-011-nd-0'

with open(os.path.join(dirname, os.path.basename(dirname)+'_non_numpy.json'), 'r') as f:
        non_numpy_results = json.load(f)
        config_dict = non_numpy_results['config_dict']
        special_list = non_numpy_results['special_list']
        f.close()
    
counts = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_counts.txt'))
config_codes = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_config_codes.txt'))
hop_list = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_hop_list.txt'))


counts = np.asarray(counts, dtype=np.int16)
config_codes = np.asarray(config_codes, dtype=np.int8)
hop_list = np.asarray(hop_list, dtype=np.int8)

In [ ]:
configs = ['']*len(config_dict)
for key, value in config_dict.items():
    configs[value] = key

total_counts = np.sum(counts, axis=0)
print("In total found: ")
for ii_conf in range(total_counts.shape[0]):
    print(configs[ii_conf]+': ', total_counts[ii_conf])

In [ ]:
import pandas as pd

symbols = []
for ii_special, specials in enumerate(special_list):
    ii_snapshot = ii_special + 1
    snapshot = trajectory[ii_snapshot]
    for special_config in specials:
        symbols.append(snapshot[special_config].get_chemical_formula(mode='hill'))
        
count = pd.Series(symbols).value_counts()
print(count)

In [ ]:
target_timesteps = []
target_specials = []

count_special = 0
for ts, special_config in enumerate(special_list):
    for specials in special_config:
        if specials:
            if hop_list[count_special] == 1:
                target_timesteps.append(ts)
                target_specials.append(specials)
                print((10*ts)+1000, count_special)
                print(specials)
            count_special += 1

print(count_special, len(hop_list))

In [ ]:

%matplotlib auto
tar = 8
print("Timestep: ", target_timesteps[tar]+1)
print("Special O: ", target_specials[tar][-1])
config_plot.configPlotter.plot_snapshot(trajectory[target_timesteps[tar]+1], target_specials[tar][-1], target_specials[tar][:-1])
plt.show()

In [ ]:
%matplotlib inline
print(np.min(hop_list), np.max(hop_list))
hop_counts = np.bincount(hop_list-np.min(hop_list))
print(hop_counts)

#-8, -7, -6, -5 ... -1
labels = ["Exchange of H?", "H ends at\nHOTi->H2O Ti", "H ends at\nHOTi->H2O Ti", "No change at site", "Surface Hopping", "Wrong Oxygen Observed", "H20 close to HOTi", "OTi gaining H", "OTi losing H"]
labels += np.arange(0, np.max(hop_list)+1).tolist()

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_title("Analysis of Ti hopping for H2O Splitting")
ax.set_xlabel('Error Codes \ Number of Hops')
ax.set_ylabel('# of occurences')

ax.bar(np.arange(np.min(hop_list), np.max(hop_list)+1), hop_counts)

ax.set_xticks(
    np.arange(np.min(hop_list), np.max(hop_list)+1),
    labels[-(np.max(hop_list) + 1 - np.min(hop_list)):],
    rotation=-45, ha="left"
)
plt.show()

In [ ]:
%matplotlib inline
plot_counts = np.bincount(hop_list-np.min(hop_list))
non_zero = np.argwhere(plot_counts!=0)

plot_counts[-3] = plot_counts[-2] + plot_counts[-3]
plot_counts = np.delete(plot_counts, [-2])
labels = ["H ends at\nHOTi->H2O Ti", "H20 close to HOTi", "H20 close to OTi"]
labels += np.arange(0, np.max(hop_list)+1).tolist()
plot_counts = plot_counts[plot_counts!=0]

structure = "Anatase 101"

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_title("Process for TiO losing or gaining an H: %s"%structure)
ax.set_xlabel('Process / Number of H20 before H bonds to TiO')
ax.set_ylabel('# of occurences')

ax.bar(np.arange(0, len(plot_counts)), plot_counts)

ax.set_xticks(
    np.arange(0, len(plot_counts)),
    labels,
    rotation=-45, ha="center"
    )
plt.tight_layout()
fig.savefig("%s_bars.pdf"%structure, format='pdf')
plt.show()

In [ ]:
draw_codes = config_codes.copy()[1:, :]

water_bound = np.array([0, 3, 4], dtype=np.int8)
bulk_bound = np.array([1, 2], dtype=np.int8)

wb_tf = np.any(draw_codes[..., np.newaxis] == water_bound[np.newaxis, np.newaxis, :], axis=-1)

# Filter out oxygen that never changes state
has_changed = np.any(draw_codes != draw_codes[0, :], axis=0)
always_waterbound = np.all(np.any(draw_codes[..., np.newaxis] == water_bound[np.newaxis, np.newaxis, :], axis=-1), axis=0)
always_bulk = np.all(np.any(draw_codes[..., np.newaxis] == bulk_bound[np.newaxis, np.newaxis, :], axis=-1), axis=0)
always_relevant = np.logical_not(np.logical_or(always_waterbound, always_bulk))

particle_selection = np.logical_and(has_changed, always_relevant)

draw_codes = draw_codes[:, particle_selection]
draw_codes = draw_codes[:, :]

norm = mpl.colors.Normalize(vmin=np.min([v for v in config_dict.values()]), vmax=np.max([v for v in config_dict.values()]))
fig, ax = plt.subplots(1, 1, figsize=(25, 6))
im = ax.imshow(draw_codes.T, aspect='auto', norm=norm, cmap="tab10", interpolation='nearest')
ax.set_title('Time development of Active Oxygen Sites')
ax.set_ylabel("Active Oxygen Index")
ax.set_xlabel('timestep')
ax.set_xlim([0, draw_codes.shape[0]-1])
cb = fig.colorbar(im, label="Configuration")
cb.ax.set_yticks(np.arange(len(config_dict)), labels=[key for key in config_dict.keys()])
fig.savefig("%s_timedev.pdf"%structure, format='pdf')
plt.show()